# Projeto de Classificaçãoo para Marinha do Brasil
### Autor: Natanael Junior (natmourajr@gmail.com)
Laboratório de Processamento de Sinais - UFRJ

Laboratório de Tecnologia Sonar

Instituto de Pesquisas da Marinha - IPqM


## Bibliotecas e leitura de dados


In [1]:
import os
import pickle
import numpy as np
import time

from sklearn.externals import joblib

init_time = time.time()

m_time = time.time()
print 'Time to import all libraries: '+str(m_time-init_time)+' seconds'

outputpath = os.environ['OUTPUTDATAPATH']
main_analysis_path = os.environ['SONAR_WORKSPACE']
log_analysis_path = os.environ['PACKAGE_OUTPUT']
result_analysis_path = os.environ['PACKAGE_OUTPUT']+'/StackedAutoEncoder'
pict_results_path = result_analysis_path+'/picts'
files_results_path = result_analysis_path+'/output_files'

# Read data
# Check if LofarData has created...
m_time = time.time()


subfolder = '4classes_old'
n_pts_fft = 1024
decimation_rate = 3

if(not os.path.exists(outputpath+'/'+'LofarData_%s_%i_fft_pts_%i_decimation_rate.jbl'%(
            subfolder,n_pts_fft,decimation_rate))):
    print outputpath+'/'+'LofarData_%s_%i_fft_pts_%i_decimation_rate.jbl'%(
        subfolder,n_pts_fft,decimation_rate)+' doesnt exist...please create it'
    exit()
    
#Read lofar data
[data,class_labels] = joblib.load(outputpath+'/'+
                                  'LofarData_%s_%i_fft_pts_%i_decimation_rate.jbl'%(
            subfolder,n_pts_fft,decimation_rate))
m_time = time.time()-m_time
print 'Time to read data file: '+str(m_time)+' seconds'

Time to import all libraries: 2.59876251221e-05 seconds
Time to read data file: 1.52421379089 seconds


## Definições do treinamento

Nessa célula temos os parâmetros do treinamento a ser realizado. No log, deve ficar armazenada a data do treinamento para a reconstrução dos resultados

In [15]:
%time

from sklearn import cross_validation

from Functions import LogFunctions as log

# Create a entry in log file
m_log = log.LogInformation()
date = m_log.CreateLogEntry("Classification",'ConvNeuralNetwork')

# Create a train information file
n_folds = 2
n_inits = 1
norm = 'mapstd'
qtd_window_in_event = 10

train_info = {}
train_info['n_folds'] = n_folds
train_info['n_inits'] = n_inits
train_info['norm'] = norm
train_info['qtd_window_in_event'] = qtd_window_in_event

# divide data in train and test for novelty detection
print 'Dividing data in trn and tst'
CVO = cross_validation.StratifiedKFold(all_trgt, n_folds)
CVO = list(CVO)
train_info['CVO'] = CVO

train_info['topologic_anal_done'] = False
train_info['train_done'] = False
train_info['results_done'] = False

train_info_name = result_analysis_path+'/train_info_files'+'/'+date+'_train_info.jbl'
joblib.dump([train_info],train_info_name,compress=9)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
Dividing data in trn and tst


['/home/natmourajr/Workspace/Doutorado/SonarAnalysis/Results/Classification/StackedAutoEncoder/train_info_files/2017_03_08_12_27_21_train_info.jbl']

In [16]:
# Read log files
from Functions import LogFunctions as log
mlog = log.LogInformation()
log_entries = mlog.RecoverLogEntries(package_name="Classification")
print log_entries

{0: {'date': '2017_03_07_23_34_47', 'package': 'StackedAutoEncoder'}, 1: {'date': '2017_03_07_23_35_21', 'package': 'StackedAutoEncoder'}, 2: {'date': '2017_03_08_00_07_14', 'package': 'StackedAutoEncoder'}, 3: {'date': '2017_03_08_00_20_36', 'package': 'StackedAutoEncoder'}, 4: {'date': '2017_03_08_11_52_41', 'package': 'StackedAutoEncoder'}, 5: {'date': '2017_03_08_11_58_10', 'package': 'StackedAutoEncoder'}, 6: {'date': '2017_03_08_12_03_24', 'package': 'ConvNeuralNetwork'}, 7: {'date': '2017_03_08_12_05_27', 'package': 'StackedAutoEncoder'}, 8: {'date': '2017_03_08_12_27_09', 'package': 'ConvNeuralNetwork'}, 9: {'date': '2017_03_08_12_27_21', 'package': 'ConvNeuralNetwork'}}


In [17]:
# Read Information of Train Info File
choose_date = '2017_03_08_12_27_21'

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'ConvNeuralNetwork':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'Analysing train performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    print 'StackedAutoEncoder Train Info File'
    print 'Date: %s'%(choose_date)
    print 'Number of Folds: %i'%(train_info['n_folds'])
    print 'Number of Inits: %i'%(train_info['n_inits'])
    print 'Line quantity in each event: %i'%(train_info['qtd_window_in_event'])
    
    if train_info['topologic_anal_done']:
        print 'Topological Analysis Done: True'
    else:
        print 'Topological Analysis Done: False'
    if train_info['train_done']:
        print 'Train Done: True'
    else:
        print 'Train Done: False'
    if train_info['results_done']:
        print 'Extract Results: True'
    else:
        print 'Extract Results: False'

Analysing train performed in 2017_03_08_12_27_21 and for ConvNeuralNetwork analysis
StackedAutoEncoder Train Info File
Date: 2017_03_08_12_27_21
Number of Folds: 2
Number of Inits: 1
Line quantity in each event: 10
Topological Analysis Done: False
Train Done: False
Extract Results: False


## Processamento dos dados

Os dados encontram-se no formato do matlab, para isso precisam ser processados para o formato de python.


In [2]:
# Process data...
# create a full data vector
all_data = {};
all_trgt = {};

for iclass, class_label in enumerate(class_labels):
    for irun in range(len(data[iclass])):
        if len(all_data) == 0:
            all_data = data[iclass][irun]['Signal']
            all_trgt = (iclass)*np.ones(data[iclass][irun]['Signal'].shape[1])
        else:
            all_data = np.append(all_data,data[iclass][irun]['Signal'],axis=1)
            all_trgt = np.append(all_trgt,(iclass)*np.ones(data[iclass][irun]
                                                           ['Signal'].shape[1]),axis=0)
            
all_data = all_data.transpose()


## Balanceamento de Classes
Os dados encontram-se desbalanceados. Com isso, os classificadores podem se especializar em uma classe (gerando mais SVs para a mesma) e não se especializar em outras

Acessados em 21/12/2016

https://svds.com/learning-imbalanced-classes/

http://www.cs.utah.edu/~piyush/teaching/ImbalancedLearning.pdf

http://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html

Para solucionar isso, a primeira solução é "criar" dados das classes com menos eventos de maneira aleatória. Outras soluções podem ser propostas posteriormente.

In [44]:
# Process data
# unbalanced data to balanced data with random data creation of small classes

# Same number of events in each class
qtd_events_biggest_class = 0
biggest_class_label = ''

for iclass, class_label in enumerate(class_labels):
    if sum(all_trgt==iclass) > qtd_events_biggest_class:
        qtd_events_biggest_class = sum(all_trgt==iclass)
        biggest_class_label = class_label
    print "Qtd event of %s is %i"%(class_label,sum(all_trgt==iclass))
print "\nBiggest class is %s with %i events"%(biggest_class_label,qtd_events_biggest_class)


balanced_data = {}
balanced_trgt = {}

from Functions import DataHandler as dh
m_datahandler = dh.DataHandlerFunctions()

for iclass, class_label in enumerate(class_labels):
    if len(balanced_data) == 0:
        class_events = all_data[all_trgt==iclass,:]
        balanced_data = m_datahandler.CreateEventsForClass(
            class_events,qtd_events_biggest_class-(len(class_events)))
        balanced_trgt = (iclass)*np.ones(qtd_events_biggest_class)
    else:
        balanced_data = np.append(balanced_data,
                                  (m_datahandler.CreateEventsForClass(
                    all_data[all_trgt==iclass,:],
                    qtd_events_biggest_class-sum(all_trgt==iclass))),
                                  axis=0)
        balanced_trgt = np.append(balanced_trgt,
                                  (iclass)*np.ones(qtd_events_biggest_class),axis=0)
        
all_data = balanced_data
all_trgt = balanced_trgt

Qtd event of ClassA is 9781
Qtd event of ClassB is 9781
Qtd event of ClassC is 9781
Qtd event of ClassD is 9781

Biggest class is ClassA with 9781 events
DataHandler Class: CreateEventsForClass
Original Size: (9781, 400)
DataHandler Class: CreateEventsForClass
Original Size: (9781, 400)
DataHandler Class: CreateEventsForClass
Original Size: (9781, 400)
DataHandler Class: CreateEventsForClass
Original Size: (9781, 400)


In [81]:
# transform a line of lofar in an "image" (sequence of lines)
new_data = np.zeros([all_data.shape[0],
                     all_data.shape[1],
                     train_info['qtd_window_in_event']])

class_count = 0
for ievent in range(all_data.shape[0]):
        if ievent > all_data.shape[0]-train_info['qtd_window_in_event']:
            continue
            
        if class_count < qtd_events_biggest_class-train_info['qtd_window_in_event']:
            new_data[ievent,:,:] = (all_data[ievent:ievent+
                                             train_info['qtd_window_in_event'],:].T)
            class_count = class_count+1
        else:
            if all_trgt[ievent-1] != all_trgt[ievent]:
                class_count = 0
                
all_window_data = new_data

## Convolutional Neural Network Analysis


First approach: Vary number of filter and their size


In [ ]:
# CNN parameters extraction

%time

from sklearn import preprocessing


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
import keras.callbacks as callbacks

from Functions import PreProcessing as preproc


models = {}
trn_descs = {}

# turn targets in sparse mode
n_classes = 4
trgt_sparse = np_utils.to_categorical(all_trgt,n_classes)

for log_id, log_entry in enumerate(log_entries):
    if log_entries[log_id]['package'] != 'ConvNeuralNetwork':
        continue
    if log_entries[log_id]['date'] != choose_date:
        continue
    print 'Analysis performed in %s and for %s analysis'%(
        log_entries[log_id]['date'],log_entries[log_id]['package'])
    
    # Read train info file
    train_info_name = '%s/train_info_files/%s_train_info.jbl'%(
        result_analysis_path,log_entries[log_id]['date'])
    
    [train_info] = joblib.load(train_info_name)
    
    # saving time
    if train_info['topologic_anal_done']:
        print 'Topological Analysis is done, just analyse it'
        continue
    
    trn_params = preproc.TrnParams(learning_rate=0.5,verbose=True,
                                   train_verbose=True, n_epochs=1)
    
    for ifold in range(train_info['n_folds']):
        train_id, test_id = train_info['CVO'][ifold]
        
        # normalização
        # como normalizar essa droga!?!?!?! Por enquanto, não normalizo!!!
        
        model = Sequential()
        model.add(Convolution1D(60, 10, border_mode='same',
                                input_shape=all_window_data.shape[1:],))
        model.add(Activation('relu'))
        model.add(Convolution1D(60, 10))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        
        model.add(Flatten())
        model.add(Dense(n_classes))
        model.add(Activation('softmax'))
        
        # Let's train the model using RMSprop
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])
        
        # Early Stopping
        earlyStopping = callbacks.EarlyStopping(monitor='val_loss',
                                                patience=25, 
                                                verbose=trn_params.train_verbose, 
                                                mode='auto')
                
        
        trn_desc = model.fit(all_window_data[train_id,:,:], trgt_sparse[train_id],
                             batch_size=trn_params.batch_size,
                             nb_epoch=trn_params.n_epochs,
                             verbose=trn_params.verbose,
                             callbacks=[earlyStopping],
                             validation_data=(all_window_data[test_id,:,:], 
                                              trgt_sparse[test_id]),
                             shuffle=True)
       
        models[ifold] = model
        trn_descs[ifold] = trn_desc
    
    print 'Train done'
    # saving file
    
    for ifold in range(train_info['n_folds']):
        model_name = (result_analysis_path+'/output_files'+'/'+
                      choose_date+'_model_fold'+str(ifold)+'.h5')
        models[ifold].save(model_name)
        
    train_info['topologic_anal_done'] = True
    joblib.dump([train_info],train_info_name,compress=9)
        


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs
Analysis performed in 2017_03_08_12_27_21 and for ConvNeuralNetwork analysis
Train on 19560 samples, validate on 19564 samples
Epoch 1/1
 5464/19560 [=======>......................] - ETA: 482s - loss: nan - acc: 0.3492